# Challenge 6: Actions
## Introduction

In this part of the challenge you will add another agent to the solution.

This time will be an agent that performs actions on behalf of the user.

## Step 1: Setup the final enviroment

## Step 2: Build on top of the previous solution

### Import the necessary libraries

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))

## Import the necessary libraries

import requests
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from urllib.parse import quote_plus 
from its_a_rag import ingestion
from sqlalchemy import create_engine
from langchain.agents import AgentExecutor, create_sql_agent, create_openai_tools_agent
from langchain_community.utilities import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.tools import tool

### Create Environment Variables

**Important:** Make sure you update your `.env` file.

In [ ]:
import os, dotenv, sys
dotenv.load_dotenv(override=True)

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))


# Setup environment
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
# Azure AI Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_FOLDER = os.getenv("AZURE_STORAGE_FOLDER")

# Local Folder for the documents
LOCAL_FOLDER = "D:\fsi2023"

# SQL Database
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_DB = os.getenv("SQL_DB")
SQL_USERNAME = os.getenv("SQL_USERNAME")
SQL_PWD = os.getenv("SQL_PWD")

# STOCK API
MOCK_API_ENDPOINT= os.getenv("MOCK_API_ENDPOINT")

# Define the questions list (if you are using your own dataset you need to change this list)
QUESTIONS = [
  "What are the revenues of GOOGLE in the year 2009?",
  "What are the revenues and the operative margins of ALPHABET Inc. in 2022 and how it compares with the previous year?",
  "Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?",
  "Can you give me the Fiscal Year 2023 Highlights for APPLE, MICROSOFT and NVIDIA?",
  "Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.",
  "What is the value of the cumulative 5-years total return of ALPHABET Class A at December 2022?",
  "What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?",
  "Can you buy 10 shares of APPLE for me?"
  ]

# Define the System prompt (you need to update this is you are using your own dataset.)
system_prompt_RAG = """ You are a financial assistant tasked with answering questions related to the financial results of major technology companies listed on NASDAQ, \n
specifically Microsoft (MSFT), Alphabet Inc. (GOOGL), Nvidia (NVDA), Apple Inc. (AAPL), and Amazon (AMZN). \n
if you don't find the answer in the context, just say `I don't know.`"""

system_prompt_START = """
  You are an agent that needs analyze the user question. \n
  Question : {input} \n
  if the question is related to stock prices answer with "stock". \n
  if the question is related to information about financial results answer with "rag". \n
  if the question is unclear or you cannot decide answer with "rag". \n
  only answer with one of the word provided.
  Your answer (stock/rag):
  """
system_prompt_SQL = """
  You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about stock prices.
  If you can't find the answer, say 'I am unable to find the answer.'
  """

## Step 1: Build on top of the previous solution

In [ ]:
# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_retries=2
)

In [ ]:
# Create the Agent State Class to store the messages between the agents
# this should include the input, output and decision as strings
class AgentState(TypedDict):
    input: str
    output: str
    decision: str
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
# Stock Agent - This agent should be copied from challenge 5

def stock_agent(state):
    # Create the DB Connection
    engine = create_engine(f"mssql+pymssql://{SQL_USERNAME}:{SQL_PWD}@{SQL_SERVER}:1433/{SQL_DB}")
    # Create the SQL Database Object and the SQL Database Toolkit Object to be used by the agent.
    db = SQLDatabase(engine=engine)
    # Create the agent using the Langhcain SQL Agent Class (create_sql_agent)
    # Structure the final prompt from the ChatPromptTemplate
    # Prepare the response using the invoke method of the agent
    # Return the response for the next agent (output and input required coming fron the Agent State)
    return {"output": response['output'], "input": state["input"]}

In [ ]:
## Node rag - This RAG agent should be copied from challenge 5

def rag_agent(state):


### Step 2: Create the Stock Action Agent

In [ ]:
## Node stock_action
# Define the tool using the @tool decorator
@tool
def buy_stocks(quantity: int, symbol: str) -> str:


@tool
def sell_stocks(quantity: int, symbol: str) -> str:


# You can combine multiple tools in a single block called Toolkit

def stock_action(state):
    # Define the LLM
    # Prepare the prompt for the agent (the create_openai_tools_agent function requires a ChatPromptTemplate object)
    # Prompt Example: "You are an agent that helps to acquire stocks. Use your tools to perform the requested action. \n if you can't perform the action, say 'I am unable to perform the action.' \n"
    # Construct the OpenAI Tools Agent
    # Prepare the Agent Executor
    # prepare the response using the invoke method of the agent
    # Return the response for the next agent (output and input required coming from the Agent State)
    
    

### Step 3: Update the Start Agent adding an addition decision answer

In [ ]:
# Create the start_agent that analyze the user question and decide if the question is related to stock prices or financial results - this is an updated from what you created in challenge 5
def start_agent(state):
    # Import the global llm
    # Prepare the prompt for the agent
    # Prepare the chain to be executed
    # Invoke the chain
    # Take the decision from the response
    # Return the response for the next agent (decision and input required coming fron the Agent State)


### Step 4: Update the previous graph to include the new agent

In [ ]:
# Create the Agent State Class to store the messages between the agents
# this should include the input, output and decision as strings
class AgentState(TypedDict):
    input: str
    output: str
    decision: str

# Create the 3 steps graph that is going to be working in the bellow "decision" condition
# Add nodes (start_agent, stock_agent, rag_agent) and conditional edges where the decision with be stock or rag
def create_graph():
    # Create the Workflow as StateGraph using the AgentState
    # Add the nodes (start_agent, stock_agent, rag_agent)
    # Add the conditional edge from start -> lamba (decision) -> stock_agent or rag_agent
    # Set the workflow entry point
    # Add the final edges to the END node
    # Compile and return the workflow


### Step 5: Test the Solution

In [ ]:
## Test Solution

# intantiate the graph (create_graph)
graph = create_graph()

# Use the graph invoke to answer the questions
# Test the graph with various questions

for QUESTION in QUESTIONS:
    print (QUESTION)
    result = graph.invoke({"input": QUESTION})
    print(result["output"])
    print ("------------------")